In [1]:
from io import BytesIO
import base64
from PIL import Image

In [2]:
DET_PATH = "pretrain_data_examples/detection_examples.tsv"
OUT_PATH = "outputs/detection_examples.tsv"

In [23]:
offset = 0
fp = open(DET_PATH, "r")
lineid_to_offset = []
for line in fp:
    lineid_to_offset.append(offset)
    offset += len(line.encode('utf-8'))
fp = open(DET_PATH, "r")
for line in fp:   
    print(line.encode('utf-8'))
    break

b'1\...\t505.856,189.994,799.744,450.016,/m/07j7r,tree&&753.664,384.00600000000003,827.392,446.572,/m/0c9ph5,flower&&61.44,175.07,103.424,209.51,/m/0c9ph5,flower&&979.968,260.596,1012.736,301.35,/m/0c9ph5,flower&&22.528,0.0,68.608,28.126,/m/0c9ph5,flower&&944.128,227.87800000000001,971.776,261.744,/m/0c9ph5,flower&&917.504,284.13,953.344,304.22,/m/0c9ph5,flower&&11.264,82.65599999999999,28.672,111.35600000000001,/m/0c9ph5,flower\n'


In [24]:
targets = ""
for j, offset in enumerate(lineid_to_offset):
    targets += f"{j+1}\t"
    fp.seek(offset)
    image_id, image, label = fp.readline().rstrip("\n").split("\t")
    image = Image.open(BytesIO(base64.urlsafe_b64decode(image))).convert("RGB")
    
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue())
    targets += f'{img_str.decode("utf-8")}\t'

    boxes_target = {"boxes": [], "ids": [],"labels": [], "area": []}
    label_list = label.strip().split('&&')
    for label in label_list:
        x0, y0, x1, y1, cat_id, cat = label.strip().split(',', 5)
        x0, y0, x1, y1 = float(x0), float(y0), float(x1), float(y1)
        boxes_target["boxes"].append([x0, y0, x1, y1])
        boxes_target["ids"].append(cat_id)
        boxes_target["labels"].append(cat)
    
    for i, (bbox, cat_id, cat) in enumerate(zip(boxes_target["boxes"], boxes_target["ids"], boxes_target["labels"])):
        target = ",".join(map(str, bbox))
        target += "," + str(cat_id)
        target += "," + str(cat)
        if i != len(boxes_target["boxes"])-1:
            target += "&&"
        targets += target
    targets += "\n"
    
with open(OUT_PATH, "w", encoding='utf-8') as f:
    f.write(targets)

In [20]:
offset = 0
fp = open(OUT_PATH, "r")
lineid_to_offset = []
for line in fp:
    lineid_to_offset.append(offset)
    print(line.encode('utf-8'))
    break

b"1\tb'...'\t505.856,189.994,799.744,450.016,/m/07j7r,tree&&753.664,384.00600000000003,827.392,446.572,/m/0c9ph5,flower&&61.44,175.07,103.424,209.51,/m/0c9ph5,flower&&979.968,260.596,1012.736,301.35,/m/0c9ph5,flower&&22.528,0.0,68.608,28.126,/m/0c9ph5,flower&&944.128,227.87800000000001,971.776,261.744,/m/0c9ph5,flower&&917.504,284.13,953.344,304.22,/m/0c9ph5,flower&&11.264,82.65599999999999,28.672,111.35600000000001,/m/0c9ph5,flower\n"
